# SQL II

Exploring advanced SQL syntax.

### Loading the Data
In this lecture, we'll continue our work with the `Dish` table. In the cells below, we connect to the database and query the table.

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
import sqlalchemy
import pandas as pd

# Create the engine
engine = sqlalchemy.create_engine("sqlite:////content/basic_examples.db")

# Establish the connection
connection = engine.connect()


**Question**: Query the entire **Dish** table.

In [ ]:
pd.read_sql("SELECT * FROM Dish", connection)

,name,type,cost
0,ravioli,entree,10
1,ramen,entree,7
2,taco,entree,7
3,edamame,appetizer,4
4,fries,appetizer,4
5,potsticker,appetizer,4
6,ice cream,dessert,5


### Filtering Groups Using `HAVING`

**Question**: Query the total dishes of each type having a maximum cost of less than 8.

In [ ]:
pd.read_sql("SELECT type, COUNT(*) as total_dishes FROM dish GROUP BY type HAVING MAX(cost) < 8; ", connection)

,type,total_dishes
0,appetizer,3
1,dessert,1


### EDA in SQL

Our typical workflow when working with "big data" is:
* Use SQL to query data from a database
* Use Python (with `pandas`) to analyze this data in detail

We can, however, still perform simple data cleaning and re-structuring using SQL directly. To do so, we'll consider the `Title` table from the IMDB dataset. We use random ordering here to get a "snapshot" of representative rows sampled from throughout the table.

In [ ]:
import sqlalchemy
import pandas as pd

# Create the engine
engine = sqlalchemy.create_engine("sqlite:////content/imdbmini.db")

# Establish the connection
connection = engine.connect()

In [ ]:
pd.read_sql("SELECT * FROM Title ORDER BY RANDOM() LIMIT 10;", connection)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,416496,movie,Bandidas,Bandidas,0,2006,None,93,"Action,Comedy,Crime"
1,106387,movie,Benny & Joon,Benny & Joon,0,1993,None,98,"Comedy,Drama,Romance"
2,1285016,movie,The Social Network,The Social Network,0,2010,None,120,"Biography,Drama"
3,1512685,movie,Los ojos de Julia,Los ojos de Julia,0,2010,None,118,"Horror,Mystery,Thriller"
4,77523,movie,Every Which Way but Loose,Every Which Way but Loose,0,1978,None,114,"Action,Comedy"
5,340855,movie,Monster,Monster,0,2003,None,109,"Biography,Crime,Drama"
6,246578,movie,Donnie Darko,Donnie Darko,0,2001,None,113,"Drama,Mystery,Sci-Fi"
7,1210819,movie,The Lone Ranger,The Lone Ranger,0,2013,None,150,"Action,Adventure,Western"
8,4189022,tvSeries,Ash vs Evil Dead,Ash vs Evil Dead,0,2015,2018,30,"Action,Comedy,Horror"
9,1848902,tvMovie,Game Change,Game Change,0,2012,None,118,"Biography,Drama,History"


#### Matching Text Using `LIKE`

**Question**: Query the title types and primary title names with the primary title including the phrase "Star Wars".

In [ ]:
pd.read_sql("SELECT titleType, primaryTitle FROM Title WHERE primaryTitle LIKE '%Star Wars%'", connection)

,titleType,primaryTitle
0,movie,Star Wars: Episode IV - A New Hope
1,movie,Star Wars: Episode V - The Empire Strikes Back
2,movie,Star Wars: Episode VI - Return of the Jedi
3,movie,Star Wars: Episode I - The Phantom Menace
4,movie,Star Wars: Episode II - Attack of the Clones
5,movie,Star Wars: Episode III - Revenge of the Sith
6,tvSeries,Star Wars: Clone Wars
7,tvSeries,Star Wars: The Clone Wars
8,movie,Star Wars: The Clone Wars
9,movie,Star Wars: Episode VII - The Force Awakens


_ means “look for exactly 1 character”

In [ ]:
pd.read_sql("SELECT titleType, primaryTitle FROM Title WHERE primaryTitle LIKE 'Harry Potter and the Deathly Hallows: Part _'", connection)


,titleType,primaryTitle
0,movie,Harry Potter and the Deathly Hallows: Part 1
1,movie,Harry Potter and the Deathly Hallows: Part 2


#### Converting Data Types Using `CAST`

**Question**: Query the primary title and runtime (cast as integer) of any 10 movies.

In [ ]:
pd.read_sql("SELECT primaryTitle, CAST(runtimeMinutes AS INT) FROM Title;", connection)

,primaryTitle,CAST(runtimeMinutes AS INT)
0,A Trip to the Moon,13.0
1,The Birth of a Nation,195.0
2,The Cabinet of Dr. Caligari,76.0
3,The Kid,68.0
4,Nosferatu,94.0
...,...,...
7569,Hollywood,347.0
7570,Holidate,104.0
7571,I Care a Lot,118.0
7572,Perfect Game,24.0


### Applying Conditions With `CASE`

Here, we return a random order so we can see the various movie ages (otherwise, the top few entries happen to all be old movies).

**Question**: Classify each movie title as 'new' if it was released before 1950 and 'mid-aged' if was released before 2000; label this column "movie_age". Select "title_type", "startYear" and "movie_age" in your query.  

In [ ]:
pd.read_sql('SELECT titleType, startYear, CASE WHEN startYear < 1950 THEN "old" WHEN startYear < 2000 THEN "mid-aged" ELSE "new" END AS movie_age FROM Title;', connection)

,titleType,startYear,movie_age
0,short,1902,old
1,movie,1915,old
2,movie,1920,old
3,movie,1921,old
4,movie,1922,old
...,...,...,...
7569,tvMiniSeries,2020,new
7570,movie,2020,new
7571,movie,2020,new
7572,tvEpisode,2019,new


### Joining Tables

We combine data from multiple tables by performing a **join**. We will explore joins using the cats database, which includes two tables: `s` and `t`.

In [ ]:
import sqlalchemy
import pandas as pd

# Create the engine
engine = sqlalchemy.create_engine("sqlite:////content/basic_examples.db")

# Establish the connection
connection = engine.connect()

In [ ]:
pd.read_sql("SELECT * FROM s", connection)

,id,name
0,0,Apricot
1,1,Boots
2,2,Cally
3,4,Eugene


In [ ]:
pd.read_sql("SELECT * FROM t", connection)

,id,breed
0,1,persian
1,2,ragdoll
2,4,bengal
3,5,persian


#### Inner Join

**Question**: Perform inner join on tables **s** and **t**.

In [ ]:
pd.read_sql("SELECT * FROM s INNER JOIN t ON s.id = t.id", connection)

,id,name,id,breed
0,1,Boots,1,persian
1,2,Cally,2,ragdoll
2,4,Eugene,4,bengal


By default, `JOIN`ing without specifying a join type will default to an inner join.

**Question**: Perform inner join on tables **s** and **t** without specifying a join type.

In [ ]:
pd.read_sql("SELECT * FROM s JOIN t ON s.id = t.id", connection)

,id,name,id,breed
0,1,Boots,1,persian
1,2,Cally,2,ragdoll
2,4,Eugene,4,bengal


### Cross Join

**Question**: Query every possible combination of rows across tables **s** and **t**.

In [ ]:
pd.read_sql("SELECT * FROM s Cross JOIN t", connection)

,id,name,id,breed
0,0,Apricot,1,persian
1,0,Apricot,2,ragdoll
2,0,Apricot,4,bengal
3,0,Apricot,5,persian
4,1,Boots,1,persian
5,1,Boots,2,ragdoll
6,1,Boots,4,bengal
7,1,Boots,5,persian
8,2,Cally,1,persian
9,2,Cally,2,ragdoll


Conceptually, an inner join is equivalent to a cross join where irrelevant rows are removed.

**Question**: Perform inner join using cross join on tables **s** and **t**.

In [ ]:
pd.read_sql("SELECT * FROM s Cross JOIN t ON s.id = t.id", connection)

,id,name,id,breed
0,1,Boots,1,persian
1,2,Cally,2,ragdoll
2,4,Eugene,4,bengal


#### Left Outer Join

**Question**: Perform left outer join on tables **s** and **t**.

In [ ]:
pd.read_sql("SELECT * FROM s Left JOIN t ON s.id = t.id", connection)

,id,name,id,breed
0,0,Apricot,NaN,None
1,1,Boots,1.0,persian
2,2,Cally,2.0,ragdoll
3,4,Eugene,4.0,bengal


#### Right Outer Join

**Question**: Perform right outer join on tables **s** and **t**.

In [ ]:
query = """
SELECT t.*, s.*
FROM t
LEFT JOIN s ON s.id = t.id

UNION ALL

SELECT t.*, s.*
FROM s
LEFT JOIN t ON s.id = t.id
WHERE t.id IS NULL
"""

# Execute the query
result = pd.read_sql(query, connection)


#### Full Outer Join

**Question**: Perform full outer join on tables **s** and **t**.

In [ ]:
import pandas as pd

query = """
SELECT s.*, t.*
FROM s
LEFT JOIN t ON s.id = t.id

UNION ALL

SELECT s.*, t.*
FROM t
LEFT JOIN s ON s.id = t.id
WHERE s.id IS NULL
"""

# Execute the query
result = pd.read_sql(query, connection)


#### Aliasing in Joins

Let's return to the IMDB dataset. Now, we'll consider two tables: `Title` and `Rating`.

In [ ]:
import sqlalchemy
import pandas as pd

# Create the engine
engine = sqlalchemy.create_engine("sqlite:////content/imdbmini.db")

# Establish the connection
connection = engine.connect()

When working with tables that have long names, we often create an **alias** using the `AS` keyword (much like we did with columns in the previous lecture). This makes it easier to reference these tables when performing a join.

**Question**: Perform inner join on tables **Title** (alias T) and **Rating** (alias R).

In [ ]:
pd.read_sql("SELECT primaryTitle, averageRating FROM Title AS T INNER JOIN Rating AS R ON T.tconst = R.tconst;", connection)

,primaryTitle,averageRating
0,A Trip to the Moon,8.2
1,The Birth of a Nation,6.3
2,The Cabinet of Dr. Caligari,8.1
3,The Kid,8.3
4,Nosferatu,7.9
...,...,...
7569,Hollywood,7.5
7570,Holidate,6.1
7571,I Care a Lot,6.3
7572,Perfect Game,9.9


Referencing columns using the full or aliased table name is important to avoid ambiguity. Suppose the tables we are trying to join both include a column with the same name, like the `tconst` columns present in both the `Title` and `Rating` tables of the IMDB database. If we do not specify which table's column we wish to reference, SQL will not be able to process our query.

In the cell below, it is unclear if we are referring to the `tconst` column from the `Title` table or the `tconst` column from the `Rating` table. SQL errors.

In [ ]:
pd.read_sql("SELECT primaryTitle, averageRating FROM Title AS T INNER JOIN Rating AS R ON tconst = tconst;", connection)


OperationalError: (sqlite3.OperationalError) ambiguous column name: tconst
[SQL: SELECT primaryTitle, averageRating FROM Title AS T INNER JOIN Rating AS R ON tconst = tconst;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)